# Landsat Collection 2 Preprocessing

Code to preprocess Landsat Collection 2 surface reflectance imagery. Please report any bugs to Christopher Kibler (kibler@ucsb.edu).

## Load Packages

In [ ]:
import numpy as np
import rasterio
import glob
import os

## Define Functions

Build mask from Landat QA pixel band

In [ ]:
def Landsat_C2_build_qa_pixel_mask(qa_pixel_img):
    for q in [0, 2, 3, 4]: #List of QA bits that are not accepted
        qa_pixel_img[np.bitwise_and(np.left_shift(1, q), qa_pixel_img) != 0] = 0
    qa_pixel_img[qa_pixel_img != 0] = 255
    return qa_pixel_img

Preprocess Landsat Collection 2 surface reflectance imagery

In [ ]:
def Landsat_C2_SR_preprocess(directory):
    
    #Load pixel QA band
    with rasterio.open(glob.glob(directory + '/L*QA_PIXEL.tif')[0]) as img:
        qa_pixel = img.read(1)
        qa_pixel = Landsat_C2_build_qa_pixel_mask(qa_pixel)

    #Load radiometric QA band and save metadata
    with rasterio.open(glob.glob(directory + '/L*QA_RADSAT.tif')[0]) as img:
        qa_radsat = img.read(1)
        landsat_img_meta = img.meta

    #Get file names for surface reflectance bands
    SR_bands = glob.glob(directory + '/LC08*SR_B[2-7].tif') + glob.glob(directory + '/LE07*SR_B[1-7].tif') + glob.glob(directory + '/LT05*SR_B[1-7].tif')
    
    #Make sure that dataset contains the right number of bands
    if len(SR_bands) != 6:
        raise ValueError('Surface reflectance dataset does not contain six bands.')
    
    #Load bands indidivually and apply preprocessing corrections
    arrs = []
    for band in SR_bands:
        with rasterio.open(band) as f:
            layer = f.read(1)
            layer = layer * 0.0000275 + (-0.2)
            layer[qa_pixel == 0] = np.nan
            layer[qa_radsat != 0] = np.nan
            layer[layer > 1] = np.nan
            layer[layer < 0] = np.nan
            arrs.append(layer)
        
    # Convert the list of bands to a single numpy array
    landsat_img = np.array(arrs, dtype=arrs[0].dtype)
    
    #Update metadata
    landsat_img_meta.update(dtype = landsat_img.dtype, count = len(SR_bands))
    
    #Create image identifier
    img_att = os.path.basename(glob.glob(directory + '/L*QA_PIXEL.tif')[0]).split("_")
    landsat_img_id = "_".join(img_att[0:7])
    
    return(landsat_img, landsat_img_meta, landsat_img_id)

Preprocess Landsat Collection 2 land surface temperature imagery

In [ ]:
def Landsat_C2_ST_preprocess(directory):    
    
    #Load pixel QA band and save metadata
    with rasterio.open(glob.glob(directory + '/L*QA_PIXEL.tif')[0]) as img:
        qa_pixel = img.read(1)
        qa_pixel = Landsat_C2_build_qa_pixel_mask(qa_pixel)
        landsat_lst_meta = img.meta
    
    #Get file name for LST band
    LST_band = glob.glob(directory + '/LC08*ST_B10.tif') + glob.glob(directory + '/LE07*ST_B6.tif') + glob.glob(directory + '/LT05*ST_B6.tif')
    
    #Make sure that dataset contains the right number of bands
    if len(LST_band) != 1:
        raise ValueError('Land surface temperature dataset does not contain one band.')
    
    #Load LST band
    with rasterio.open(LST_band[0]) as img:
        landsat_lst = img.read(1)
        landsat_lst =  landsat_lst * 0.00341802 + 149
        landsat_lst[qa_pixel == 0] = np.nan
    
    #Update metadata
    landsat_lst_meta.update(dtype = landsat_lst.dtype, count = 1)
    
    #Create image identifier
    img_att = os.path.basename(glob.glob(directory + '/L*QA_PIXEL.tif')[0]).split("_")
    landsat_img_id = "_".join(img_att[0:7])
    
    return landsat_lst, landsat_lst_meta, landsat_img_id

## Execute Functions and Export Results

In [ ]:
#Directory that contains the image files for the individual bands downloaded from Earth Explorer
directory = 'Images/LC08_L2SP_035038_20200607_20200824_02_T1'

vswir, vswir_meta, vswir_id = Landsat_C2_SR_preprocess(directory)

with rasterio.open(directory + '/' + vswir_id + '_vswir.tif', 'w', **vswir_meta) as dst:
    dst.write(vswir)

thermal, thermal_meta, thermal_id = Landsat_C2_ST_preprocess(directory)

with rasterio.open(directory + '/' + thermal_id + '_thermal.tif', 'w', **thermal_meta) as dst:
    dst.write(thermal, 1)